In [30]:
from glob import glob
import scipy.io
import numpy as np
import mne
import copy
from sklearn.model_selection import StratifiedKFold, train_test_split
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import torch.nn.functional as F
import time as t
import sklearn.metrics as metrics
from scipy import signal
from sklearn import preprocessing

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
torch.cuda.empty_cache()

In [482]:
rest = "/home/cbme/phd/bmz198612/EEG/EEG_Subject/S22/Rest"
easy = "/home/cbme/phd/bmz198612/EEG/EEG_Subject/S22/Easy"
med = "/home/cbme/phd/bmz198612/EEG/EEG_Subject/S22/Med"
hard = "/home/cbme/phd/bmz198612/EEG/EEG_Subject/S22/Hard"

In [483]:
for temp in glob(easy+"/*.mat"):
    data=scipy.io.loadmat(temp)
    print(temp)
    temp=temp.replace(easy,"")
    temp=temp.replace('.mat','')
    temp=temp.replace('/','')
    print(temp)
    break

/home/cbme/phd/bmz198612/EEG/EEG_Subject/S22/Easy/easy12nbStr2S22.mat
easy12nbStr2S22


In [484]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'easy12nbStr2S22'])

In [485]:
data = data[temp]
data.shape

(63, 12000)

In [486]:
num_chan = 63
sampling_freq = 200
ch_types = ['eeg']*63
info = mne.create_info(num_chan,sampling_freq)
info.set_montage('standard_1020')


<Info | 8 non-empty values
 bads: []
 ch_names: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, ...
 chs: 63 misc
 custom_ref_applied: False
 dig: 3 items (3 Cardinal)
 highpass: 0.0 Hz
 lowpass: 100.0 Hz
 meas_date: unspecified
 nchan: 63
 projs: []
 sfreq: 200.0 Hz
>

In [487]:
data = mne.io.RawArray(data,info)

Creating RawArray with float64 data, n_channels=63, n_times=12000
    Range : 0 ... 11999 =      0.000 ...    59.995 secs
Ready.


In [488]:
epochs = mne.make_fixed_length_epochs(data,duration=3,overlap=1.5)

Not setting metadata
39 matching events found
No baseline correction applied
0 projection items activated


In [489]:
epochs.get_data().shape

Using data from preloaded Raw for 39 events and 600 original time points ...
0 bad epochs dropped


(39, 63, 600)

In [490]:



rest_data = []
stress_data = []





In [491]:
def convertmat2mne(data):
    ch_types=['eeg']*63
    ch_names=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62']    
    info = mne.create_info(ch_names,ch_types=ch_types,sfreq=200)
    data = mne.io.RawArray(data,info)
    data.set_eeg_reference()
    epochs=mne.make_fixed_length_epochs(data,duration=3,overlap=2.4)
    return epochs.get_data()

In [492]:
%%capture
from glob import glob
import scipy.io
for temp in glob(rest+"/*.mat"):
    data=scipy.io.loadmat(temp)
    index=temp.replace(rest,"")
    index=index.replace('.mat','')
    index=index.replace('/','')
    data=data[index]
    data = convertmat2mne(data)
    rest_data.append(data)

In [493]:
%%capture
from glob import glob
import scipy.io
for temp in glob(easy+"/*.mat"):
    data=scipy.io.loadmat(temp)
    index=temp.replace(easy,"")
    index=index.replace('.mat','')
    index=index.replace('/','')
    data=data[index]
    data = convertmat2mne(data)
    stress_data.append(data)
        

In [494]:
%%capture
from glob import glob
import scipy.io
for temp in glob(med+"/*.mat"):
    data=scipy.io.loadmat(temp)
    index=temp.replace(med,"")
    index=index.replace('.mat','')
    index=index.replace('/','')
    data=data[index]
    data = convertmat2mne(data)
    stress_data.append(data)

In [495]:
%%capture
from glob import glob
import scipy.io
for temp in glob(hard+"/*.mat"):
    data=scipy.io.loadmat(temp)
    index=temp.replace(hard,"")
    index=index.replace('.mat','')
    index=index.replace('/','')
    data=data[index]
    data = convertmat2mne(data)
    stress_data.append(data)

In [496]:
rest_epochs_labels = [len(i)*[0] for i in rest_data]
stress_epochs_labels = [len(i)*[1] for i in stress_data]



In [497]:
data_list = rest_data+stress_data
label_list = rest_epochs_labels+stress_epochs_labels
print(len(data_list),len(label_list))


15 15


In [498]:
groups_list = [[i]*len(j) for i ,j in enumerate(data_list)]


In [499]:
len(groups_list[1])

96

In [500]:
data_array = np.vstack(data_list)
label_array = np.hstack(label_list)
group_array = np.hstack(groups_list)
data_array = np.moveaxis(data_array,1,2)

print(data_array.shape,label_array.shape)

(1440, 600, 63) (1440,)


In [501]:
from sklearn.model_selection import StratifiedGroupKFold,GroupKFold,LeaveOneGroupOut,GroupShuffleSplit
from sklearn.preprocessing import StandardScaler

from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.preprocessing import StandardScaler
#https://stackoverflow.com/questions/50125844/how-to-standard-scale-a-3d-matrix
class StandardScaler3D(BaseEstimator,TransformerMixin):
    #batch, sequence, channels
    def __init__(self):
        self.scaler = StandardScaler()

    def fit(self,X,y=None):
        self.scaler.fit(X.reshape(-1, X.shape[2]))
        return self

    def transform(self,X):
        return self.scaler.transform(X.reshape( -1,X.shape[2])).reshape(X.shape)

In [502]:
data_array = data_array.reshape((data_array.shape[0],1,data_array.shape[1],data_array.shape[2]))

In [503]:
data_array[1].shape

(1, 600, 63)

In [504]:
data_array.shape

(1440, 1, 600, 63)

In [505]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()
   

In [506]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(in_channels=1,out_channels=32,kernel_size=(1,25),stride=1)
        self.conv2=nn.Conv2d(in_channels=32,out_channels=32,kernel_size=(63,1),stride=1)
        self.BN = nn.BatchNorm2d(32)
        self.GL = nn.GELU()
        self.AP = nn.AvgPool2d((1,50),(1,25))
        self.dropout = nn.Dropout(0.4)
        self.lstm = nn.LSTM(32,64,2,batch_first=True,dropout=0.4)
        self.linear = nn.Linear(64,2)
        self.transformer_encoder = nn.TransformerEncoder(nn.TransformerEncoderLayer(d_model=32,nhead=8,dim_feedforward=128,batch_first=True,norm_first=True),num_layers=1)
        self.mlp = nn.Sequential(
            nn.Linear(1184, 4)
        )
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x)
        x=self.BN(x)
        x=self.GL(x)
        x=self.AP(x)
        x=self.dropout(x)
        #print(x.shape)
        b,e,h,w = x.shape
        x=x.view(b,h*w,e)
        x,_=self.lstm(x)
        x=self.linear(x[:,-1,:])
        return x

In [507]:
input=torch.randn(16,1,63,600)
conformer=Model()

test_model = Model()
out=conformer(input)
out.shape

torch.Size([16, 2])

In [508]:
conformer = conformer.to(device)
test_model = test_model.to(device)

In [509]:
import matplotlib.pyplot as plt
precision=[]
accuracy=[]
recall=[]
f1=[]

In [510]:
def train(model, dataloader,train_data, device,optimizer,criterion):
    print('Training')
    model.train()
    counter = 0
    train_running_loss = 0.0
    loss_values=[]
    running_corrects = 0.0
    for batch_index, (x, y) in enumerate(dataloader):
        counter += 1
        x=x.to(device)
        y=y.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        # apply sigmoid activation to get all the outputs between 0 and 1
        y = y.to(torch.int64)
        loss = criterion(outputs, y)
        train_running_loss += loss.item()
        # backpropagation
        loss.backward()
        # update optimizer parameters
        optimizer.step()
        train_loss = train_running_loss / len(dataloader)
        loss_values.append(train_loss)
    return train_loss
                
            

In [511]:
def validate(model, dataloader,val_data, device,optimizer,criterion):
    print('validating')
    model.eval()
    counter = 0
    val_running_loss = 0.0
    with torch.no_grad():
         for batch_index, (x, y) in enumerate(dataloader):
            counter += 1
            x=x.to(device)
            y=y.to(device)
            outputs = model(x)
            y = y.to(torch.int64)
            loss = criterion(outputs, y)
            val_running_loss += loss.item()
            val_loss = val_running_loss / len(dataloader)
    return val_loss

In [512]:
def test(model, dataloader,test_data, device,optimizer,criterion):
    y_target = []
    y_pred = []
    print('Testing')
    checkpoint = torch.load('/home/cbme/phd/bmz198612/EEG/Conformer/conformer1.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    with torch.no_grad():
         for batch_index, (x, y) in enumerate(dataloader):
            x=x.to(device)
            y=y.to(device)
            outputs = model(x)
            y = y.to(torch.int64)
            _,prediction=torch.max(outputs.data,1)
            y_target.extend(y.cpu().data.numpy().tolist())
            y_pred.extend(prediction.cpu().data.numpy().tolist())
            
            
    p = metrics.precision_score(y_target,y_pred,average="macro")
    precision.append(p)
    a = metrics.accuracy_score(y_target,y_pred)
    accuracy.append(a)
    r = metrics.recall_score(y_target,y_pred,average="macro")
    recall.append(r)
    f = metrics.f1_score(y_target,y_pred,average="macro")
    f1.append(f)
    print(p)
    print(a)
    print(r)
    print(f)
         
         
     
    
    
    

In [513]:
gkf=StratifiedGroupKFold(n_splits=3)
for train_index, test_index in gkf.split(data_array, label_array, groups=group_array):
    train_features,train_labels,train_group=data_array[train_index],label_array[train_index],group_array[train_index]
    
    test_features,test_labels=data_array[test_index],label_array[test_index]
    
    scaler=StandardScaler3D()
    train_features=scaler.fit_transform(train_features)
    test_features=scaler.transform(test_features)
    train_features=np.moveaxis(train_features,2,3)
    test_features=np.moveaxis(test_features,2,3)
    train_features = torch.Tensor(train_features)
    test_features = torch.Tensor(test_features)
    train_labels = torch.Tensor(train_labels)
    test_labels = torch.Tensor(test_labels)
   
    
    X_train, X_val, y_train, y_val = train_test_split(train_features, train_labels, test_size=0.11, random_state=42)
    print(X_train.shape)
    val_data = torch.utils.data.TensorDataset(X_val, y_val)
    print(X_val.shape)
    train_data = torch.utils.data.TensorDataset(X_train, y_train)
    test_data = torch.utils.data.TensorDataset(test_features, test_labels)
    print(test_features.shape)
    train_loader = torch.utils.data.DataLoader(train_data, batch_size = 16, shuffle = False)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size = 32, shuffle = False)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size = 32, shuffle = False)
    train_loss = []
    valid_loss = []
    epochs = 50
    start = t.time()
    optimizer = optim.Adam(conformer.parameters(), lr = 0.001)
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,factor=0.1, patience=3)
    for epoch in range(epochs):
        print(f"Epoch {epoch+1} of {epochs}")
        valid_min_loss = np.Inf
        train_epoch_loss = train(conformer,train_loader,train_data,device,optimizer,criterion)
        valid_epoch_loss = validate(conformer,val_loader,val_data,device,optimizer,criterion)
        train_loss.append(train_epoch_loss)
        valid_loss.append(valid_epoch_loss)
        scheduler.step(valid_epoch_loss)
        print(f"Train Loss: {train_epoch_loss:.4f}")
        print(f'Val Loss: {valid_epoch_loss:.4f}')
        if(valid_epoch_loss <= valid_min_loss):
            valid_min_loss = valid_epoch_loss
            torch.save({
                'epoch': epochs,
                'model_state_dict': conformer.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': criterion,
                }, '/home/cbme/phd/bmz198612/EEG/Conformer/conformer1.pth')
    print(t.time()-start)
    test(test_model,test_loader,test_data,device,optimizer,criterion)
    conformer.apply(reset_weights)
        


torch.Size([854, 1, 63, 600])
torch.Size([106, 1, 63, 600])
torch.Size([480, 1, 63, 600])
Epoch 1 of 50
Training
validating
Train Loss: 0.5219
Val Loss: 0.4726
Epoch 2 of 50
Training
validating
Train Loss: 0.4712
Val Loss: 0.4703
Epoch 3 of 50
Training
validating
Train Loss: 0.4603
Val Loss: 0.4740
Epoch 4 of 50
Training
validating
Train Loss: 0.4636
Val Loss: 0.4756
Epoch 5 of 50
Training
validating
Train Loss: 0.4600
Val Loss: 0.4673
Epoch 6 of 50
Training
validating
Train Loss: 0.4593
Val Loss: 0.4710
Epoch 7 of 50
Training
validating
Train Loss: 0.4529
Val Loss: 0.4686
Epoch 8 of 50
Training
validating
Train Loss: 0.4529
Val Loss: 0.4889
Epoch 9 of 50
Training
validating
Train Loss: 0.4427
Val Loss: 0.4803
Epoch 10 of 50
Training
validating
Train Loss: 0.4143
Val Loss: 0.4124
Epoch 11 of 50
Training
validating
Train Loss: 0.3747
Val Loss: 0.3753
Epoch 12 of 50
Training
validating
Train Loss: 0.3694
Val Loss: 0.3674
Epoch 13 of 50
Training
validating
Train Loss: 0.3463
Val Loss: 0.3

validating
Train Loss: 0.1445
Val Loss: 0.0889
Epoch 50 of 50
Training
validating
Train Loss: 0.1404
Val Loss: 0.0889
20.055817127227783
Testing
0.5459110473457676
0.68125
0.5546875
0.5474460651100882
Reset trainable parameters of layer = NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
Reset trainable parameters of layer = Linear(in_features=32, out_features=128, bias=True)
Reset trainable parameters of layer = Linear(in_features=128, out_features=32, bias=True)
Reset trainable parameters of layer = LayerNorm((32,), eps=1e-05, elementwise_affine=True)
Reset trainable parameters of layer = LayerNorm((32,), eps=1e-05, elementwise_affine=True)
Reset trainable parameters of layer = Linear(in_features=1184, out_features=4, bias=True)
Reset trainable parameters of layer = Conv2d(1, 32, kernel_size=(1, 25), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(32, 32, kernel_size=(63, 1), stride=(1, 1))
Reset trainable parameters of layer = BatchNorm2d(32, ep

In [514]:
print(sum(precision)/3,sum(accuracy)/3,sum(recall)/3,sum(f1)/3)

0.614555019346985 0.7548611111111111 0.5811631944444445 0.5871950050556605


In [515]:
    
    


"""
kf = KFold(n_splits=5)
batch_size=64
for i, (train_index, test_index) in enumerate(kf.split(eeg_data)):
        ### Dividing data into folds
        print(f"Fold {i}:")
        x_train_fold = eeg_data[train_index]
        x_test_fold = eeg_data[test_index]
        y_train_fold = labels[train_index]
        y_test_fold = labels[test_index]
        X_train, X_val, y_train, y_val = train_test_split(x_train_fold, y_train_fold, test_size=0.1, random_state=42)
        val_data = torch.utils.data.TensorDataset(X_val, y_val)
        train_data = torch.utils.data.TensorDataset(X_train, y_train)
        test_data = torch.utils.data.TensorDataset(x_test_fold, y_test_fold)
        train_loader = torch.utils.data.DataLoader(train_data, batch_size = 16, shuffle = False)
        val_loader = torch.utils.data.DataLoader(val_data, batch_size = 32, shuffle = False)
        test_loader = torch.utils.data.DataLoader(test_data, batch_size = 32, shuffle = False)
        
        train_loss = []
        valid_loss = []
        epochs = 50
        start = t.time()
        optimizer = optim.Adam(conformer.parameters(), lr = 0.001)
        criterion = nn.CrossEntropyLoss()
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,factor=0.1, patience=3)
        for epoch in range(epochs):
            print(f"Epoch {epoch+1} of {epochs}")
            valid_min_loss = np.Inf
            train_epoch_loss = train(conformer,train_loader,train_data,device,optimizer,criterion)
            valid_epoch_loss = validate(conformer,val_loader,val_data,device,optimizer,criterion)
            train_loss.append(train_epoch_loss)
            valid_loss.append(valid_epoch_loss)
            scheduler.step(valid_epoch_loss)
            print(f"Train Loss: {train_epoch_loss:.4f}")
            print(f'Val Loss: {valid_epoch_loss:.4f}')
            if(valid_epoch_loss <= valid_min_loss):
                valid_min_loss = valid_epoch_loss
                torch.save({
                    'epoch': epochs,
                    'model_state_dict': conformer.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': criterion,
                    }, 'C:/Users/Anoop Godiyal/Desktop/Path/conformer.pth')
        print(t.time()-start)
        
        test(test_model,test_loader,test_data,device,optimizer,criterion)
        conformer.apply(reset_weights)
"""

'\nkf = KFold(n_splits=5)\nbatch_size=64\nfor i, (train_index, test_index) in enumerate(kf.split(eeg_data)):\n        ### Dividing data into folds\n        print(f"Fold {i}:")\n        x_train_fold = eeg_data[train_index]\n        x_test_fold = eeg_data[test_index]\n        y_train_fold = labels[train_index]\n        y_test_fold = labels[test_index]\n        X_train, X_val, y_train, y_val = train_test_split(x_train_fold, y_train_fold, test_size=0.1, random_state=42)\n        val_data = torch.utils.data.TensorDataset(X_val, y_val)\n        train_data = torch.utils.data.TensorDataset(X_train, y_train)\n        test_data = torch.utils.data.TensorDataset(x_test_fold, y_test_fold)\n        train_loader = torch.utils.data.DataLoader(train_data, batch_size = 16, shuffle = False)\n        val_loader = torch.utils.data.DataLoader(val_data, batch_size = 32, shuffle = False)\n        test_loader = torch.utils.data.DataLoader(test_data, batch_size = 32, shuffle = False)\n        \n        train_lo